In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import selenium as se
import time
from webcrawler.researchgate import IeeeCrawler, ResearchGateCrawler
import mysql.connector
import textdistance
import os

In [2]:
cnx = mysql.connector.connect(user='david', password='daviddung1993',
                              host='127.0.0.1',
                              database='computervision')
cursor = cnx.cursor()

In [3]:
ieee_crawler = IeeeCrawler()
research_crawler = ResearchGateCrawler()

In [4]:
cursor.execute("select p.PaperID, p.Title from Papers p where p.DOI is NULL")
all_paper = cursor.fetchall()
len(all_paper)

65715

In [6]:
cursor.execute("select distinct p.PaperID, p.Title from Papers p where p.journalName like '%ieee%'")
all_paper2 = cursor.fetchall()
len(all_paper2)

17246

In [ ]:
for idx, paper in enumerate(all_paper):
    if idx % 10 == 0:
        print(idx)
    title = paper[1]

    missing_affiliation = False

    cursor.execute("select distinct a.AuthorID, a.Name from Papers p,authoredBy b,Authors a where p.PaperID=%s and p.PaperID = b.PaperID and b.AuthoredByID not in (SELECT AuthorID from affiliatedTo) and b.AuthoredByID = a.AuthorID",[paper[0]])
    authors = cursor.fetchall()

    if not authors or authors[0][0] == "0":
        continue

    try:
        first_name = authors[0][1].split()[-1]
    except:
        print(authors)
        raise ValueError

    #affiliations = ieee_crawler.make_request(title, first_name)
    try:
        affiliations, doi = research_crawler.make_request(title,first_name)
    except:
        print(paper[0])
        continue
    cursor.execute("UPDATE Papers p SET p.DOI=%s where p.PaperID=%s",[doi,paper[0]])

    if not affiliations:
        continue

    for author in authors:
        try:
            jaccard_dist = [textdistance.jaccard(author[1].lower().split(), aff_auth[0].lower().split()) for aff_auth in affiliations]
            if max(jaccard_dist) == 0:
                continue
            max_similarity_index = jaccard_dist.index(max(jaccard_dist))

            affiliation = affiliations[max_similarity_index][1]
            if not affiliations[max_similarity_index][1]:
                missing_affiliation = True
                continue
            cursor.execute("SELECT * FROM Affiliations a WHERE a.Name=%s", [affiliation])
            entry = cursor.fetchone()
            if not entry:
                cursor.execute("INSERT INTO Affiliations (Name) VALUES (%s)", [affiliation[:500]])
                lastrowid = cursor.lastrowid
            else:
                lastrowid = entry[0]
            cursor.execute("INSERT INTO affiliatedTo VALUES (%s,%s)", [author[0],lastrowid])
        except AttributeError:
            continue

    #print(f"DONE")
    #time.sleep(1)

    cnx.commit()

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
06ecb4303b4d7b432c5cbee0341edc3ae3ada586
340
350
360
370
0730faa8a4c6372e1d42a7b880a42edfcb32b5fe
380
390
400
410
07993804501ae4df9fe43cf8afb009ae38fe902e
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
093869a6f8f229a6e2e9ad15cd5e666173e66456
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
0b47a063fb29d2688226a289b84120661fcdc216
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1

In [ ]:
"david"[:3]

In [ ]:
resp = requests.get("https://api.crossref.org/journals?query=Pattern Recognition").json()
resp

In [ ]:
resp["message"]["items"][2]

In [ ]:
soup = BeautifulSoup(requests.get("https://ieeexplore.ieee.org/document/9872831/authors#authors").content, 'html.parser')

In [ ]:
print(soup.prettify())

In [ ]:
classes = []
for element in soup.find_all(class_=True):
    classes.extend(element["class"])

In [ ]:
soup["header"]

In [ ]:
options = se.webdriver.ChromeOptions()
#options.add_argument('headless')
options.add_argument(
    "user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.1 Safari/605.1.15")
browser = se.webdriver.Chrome(options=options)
browser.get("https://ieeexplore.ieee.org/document/9872831/authors#authors")
source = browser.page_source

In [ ]:
soup = BeautifulSoup(source)

In [ ]:
for node in soup.find_all("div", {"class":"authors-accordion-container"}):
    #print(node.findAll(Text=True))
    print(node.findAll(text=True))

In [ ]:
cursor.execute("select distinct a.AuthorID, a.Name from Papers p,authoredBy b,Authors a where p.PaperID=%s and p.PaperID = b.PaperID and b.AuthoredByID not in (SELECT AuthorID from affiliatedTo) and b.AuthoredByID = a.AuthorID",["00388ed29b7e6882bdf66b7b963cd8231d69ec75"])
authors = cursor.fetchall()
first_name = authors[0][1].split()[-1]
affiliations = research_crawler.make_request("Evidence for horizontal gene transfer in Escherichia coli speciation.",first_name)

In [ ]:
research_crawler.browser.quit()

In [ ]:
cursor.fetchall()

In [ ]:
cursor.close()
cnx.close()

In [ ]:
textdistance.jaccard("optimal combinations of pattern classifiers".split(), "optimal combination of pattern classifiers".split())

In [ ]:
options = se.webdriver.ChromeOptions()
options.add_argument('headless')

options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.1 Safari/605.1.15")
browser = se.webdriver.Chrome(options=options)

In [ ]:
#url = "https://ieeexplore.ieee.org/search/searchresult.jsp?action=search&newsearch=true&matchBoolean=true&queryText=(%22All%20Metadata%22:Automatic%20Image%20Segmentation%20by%20Dynamic%20Region%20Merging)%20AND%20(%22Authors%22:Zhang)"
url = "https://www.researchgate.net/search/publication?q=machine learning"

browser.get(url)
#time.sleep(5)
html = browser.page_source
soup = BeautifulSoup(html)

In [ ]:
titles = []
authors = []
publications = []
for node in soup.find_all("div", {"class": "nova-legacy-v-publication-item__title"}):
    titles.append(''.join(node.findAll(text=True)))

for node in soup.find_all("ul", {"class": "nova-legacy-v-publication-item__person-list"}):
    authors.append(node.findAll(text=True))

for node in soup.find_all("a", {"class": "nova-legacy-e-link--theme-bare"}):
    publications.append(node["href"])

In [ ]:
alla = list(zip(titles,authors))
alla, publications

In [ ]:
link = f"https://www.researchgate.net/{publications[0]}"
browser.get(link)
html = browser.page_source
soup_author = BeautifulSoup(html)

In [ ]:
link

In [ ]:
authors_new = []
for node in soup_author.find_all("div", {"class": "nova-legacy-v-person-list-item__body"}):
    author_name = node.find_all("div", {"class":"nova-legacy-v-person-list-item__title"})[0].find(text=True)
    aff_element = node.find_all("li",{"class","nova-legacy-v-person-list-item__meta-item"})
    if aff_element:
        author_aff = aff_element[0].find(text=True)
    else:
        author_aff = ""
    authors_new.append((author_name, author_aff))


In [ ]:
authors_new

In [ ]:
authors_new

In [ ]:
import random
random.random() + 1

In [ ]:
jaccard_dista = [textdistance.jaccard("David Haeger".lower().split(), aff_auth.lower().split()) for aff_auth in ["David J. Fleet", "Hermann Wagner", "David J. Haeger"]]

In [ ]:
jaccard_dista